In [7]:
%reload_ext autoreload
%autoreload 2

In [12]:
import spacy

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import json

In [10]:
import sys
print(sys.version)
print(sys.executable)
print(spacy.__version__)

3.10.19 (main, Oct  9 2025, 15:25:03) [Clang 16.0.0 (clang-1600.0.26.6)]
/opt/homebrew/opt/python@3.10/bin/python3.10
3.7.5


In [ ]:
# %pip install pyarrow
# %pip install -U pyarrow pandas

In [4]:
# pip install pandas

# if you ran intp 3.9 python version problem in vscode, do:
# /opt/homebrew/bin/python3.10 -m pip install --upgrade pip
# /opt/homebrew/bin/python3.10 -m pip install ipykernel
# /opt/homebrew/bin/python3.10 -m ipykernel install --user --name=python310 --display-name "Python 3.10"

# then in top right, switch Python 3.10.*

In [ ]:
# %pip install --upgrade pip
# %pip install torch torchvision torchaudio sentence-transformers
# %pip install spacy==3.7.1
# %pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl

In [6]:
from uml_project import *
# note: constants.py requires python version >= 3.10
HF_DIR, HF_REGISTRY, MODEL_DIR # DEVICE_TORCH

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(PosixPath('/Users/alenachan/Desktop/Columbia/Fall-2025/UML/UML/data/huggingface'),
 PosixPath('/Users/alenachan/Desktop/Columbia/Fall-2025/UML/UML/data/huggingface/REGISTRY.json'),
 PosixPath('/Users/alenachan/Desktop/Columbia/Fall-2025/UML/UML/data/models'))

In [11]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("sentencizer")

In [13]:
tsne_tex_path = Path(
	"/Users/alenachan/Desktop/Columbia/Fall-2025/UML/UML/data/scientific/stochastic_neighbor_embedding_under_f_divergences__discussion.json"
)

In [14]:
tsne_tex_dict = json.loads(tsne_tex_path.read_text())

In [15]:
tsne_tex_dict.keys()

dict_keys(['authors', 'date_published', 'raw_tex', 'title'])

In [16]:
tsne_sents1 = define_sentence(tsne_tex_dict["raw_tex"], nlp=nlp, latex_mode="sentences")

In [17]:
tsne_sents1

array(['Other divergences for -SNE optimization have been explored previously.',
       'Perhaps the first detailed study was done by.',
       '(Gamma- Bregman- and -divergences) and their corresponding visualizations on.',
       'some image processing datasets.',
       'different divergences can be used to find micro and macro relationships in.\ndata.',
       ', but do not focus on faithful discovery of intrinsic structures.',
       'An interesting line of work by and.\ndiscovery and multi-scale visualizations to find local and global structures.',
       'The work by is closely related where they study -divergences.',
       'from an informational retrieval perspective.',
       'Our work extends it to the general class of -divergences and.\nexplores the relationships between data structure and the type of divergence used.',
       'It is worth emphasizing that no previous study makes an explicit connection.',
       'between the choice of divergence and the type of structure di

In [18]:
tsne_sents2 = define_sentence(tsne_tex_dict["raw_tex"], nlp=nlp, latex_mode="strip")

In [19]:
tsne_sents2

array(['Other divergences for -SNE optimization have been explored previously.',
       'Perhaps the first detailed study was done by.',
       'some image processing datasets.',
       'different divergences can be used to find micro and macro relationships in.',
       'data.', 'An interesting line of work by and.',
       'discovery and multi-scale visualizations to find local and global structures.',
       'The work by is closely related where they study -divergences.',
       'from an informational retrieval perspective.',
       'Our work extends it to the general class of -divergences and.',
       'explores the relationships between data structure and the type of divergence used.',
       'It is worth emphasizing that no previous study makes an explicit connection.',
       'between the choice of divergence and the type of structure discovery.',
       'Our work makes this explicit.',
       'and should help a practitioner gain better insights about their data in the data expl

In [20]:
with open("/Users/alenachan/Desktop/Columbia/Fall-2025/UML/UML/uml_project/data/pre_processing/tmp.txt", "r", encoding="utf-8") as f:
    list_of_paths = f.read().splitlines()

In [24]:
print(list_of_paths[:10])

list_of_paths = list_of_paths[1:] # ignore /Users/alenachan/Desktop/Columbia/Fall-2025/UML/UML/data/scientific/REGISTRY.json

['/Users/alenachan/Desktop/Columbia/Fall-2025/UML/UML/data/scientific/REGISTRY.json', '/Users/alenachan/Desktop/Columbia/Fall-2025/UML/UML/data/scientific/a_neural_network_solves_explains_and_generates_university_math_problems_by_program_synthesis_and_few_shot_learning_at_human_level__generation_18_01.json', '/Users/alenachan/Desktop/Columbia/Fall-2025/UML/UML/data/scientific/a_neural_network_solves_explains_and_generates_university_math_problems_by_program_synthesis_and_few_shot_learning_at_human_level__generation_18_02.json', '/Users/alenachan/Desktop/Columbia/Fall-2025/UML/UML/data/scientific/a_neural_network_solves_explains_and_generates_university_math_problems_by_program_synthesis_and_few_shot_learning_at_human_level__generation_18_03.json', '/Users/alenachan/Desktop/Columbia/Fall-2025/UML/UML/data/scientific/a_neural_network_solves_explains_and_generates_university_math_problems_by_program_synthesis_and_few_shot_learning_at_human_level__generation_18_05.json', '/Users/alenachan/

In [39]:
OUTPUT_DIR = "data/scientific/processed"
OUTPUT_PARQUET = os.path.join(OUTPUT_DIR, "papers.parquet")
METADATA_FILE = os.path.join(OUTPUT_DIR, "metadata.json")

os.makedirs(OUTPUT_DIR, exist_ok=True)

for pth in list_of_paths:
	# print(pth)
	doc_id = os.path.splitext(os.path.basename(pth))[0]

	tsne_tex_path = Path(pth)
	tsne_tex_dict = json.loads(tsne_tex_path.read_text())
	tsne_sents = define_sentence(tsne_tex_dict["raw_tex"], nlp=nlp, latex_mode="sentences")

	if len(tsne_sents) == 0:
		print(f"Skipping {doc_id}: tsne_sents returned empty")
		continue

	records = []

	for i, sent in enumerate(tsne_sents):
		records.append({
			"corpus": "scientific_papers",
			"doc_id": doc_id,
			"sent_index": i,
			"sentence": sent,
			"split": "train",  # could later vary by dataset
			"token_count": len(sent.split())
		})
	
	df = pd.DataFrame(records)
	table = pa.Table.from_pandas(df)
	pq.write_table(table, OUTPUT_PARQUET)

	metadata = {
		"corpus_name": "scientific_papers",
		"description": "Sentence-level dataset built from parsed scientific papers.",
		"num_documents": len(df['doc_id'].unique()),
		"num_sentences": len(df),
		"columns": df.columns.tolist(),
		"example_entry": df.iloc[0].to_dict() if len(df) else {}
	}

	with open(METADATA_FILE, "w", encoding="utf-8") as f:
		json.dump(metadata, f, indent=4)

Skipping an_analysis_of_document_graph_construction_methods_for_amr_summarization__alg_person_merge: tsne_sents returned empty
Skipping an_analysis_of_document_graph_construction_methods_for_amr_summarization__fig_example: tsne_sents returned empty
Skipping an_analysis_of_document_graph_construction_methods_for_amr_summarization__fig_pipeline: tsne_sents returned empty
Skipping an_analysis_of_document_graph_construction_methods_for_amr_summarization__tab_merge: tsne_sents returned empty
Skipping main__appendixc: tsne_sents returned empty
Skipping main__appendixd: tsne_sents returned empty
Skipping main__appendixe: tsne_sents returned empty
Skipping main__appendixg: tsne_sents returned empty
Skipping main__appendixo: tsne_sents returned empty
Skipping main__appendixp: tsne_sents returned empty
Skipping main__appendixq: tsne_sents returned empty
Skipping main__appendixr: tsne_sents returned empty
Skipping main__appendixs: tsne_sents returned empty
Skipping main__appendixt: tsne_sents ret

In [40]:
df = pd.read_parquet("/Users/alenachan/Desktop/Columbia/Fall-2025/UML/UML/data/scientific/processed/papers.parquet")
print(df.head())

              corpus                                             doc_id  \
0  scientific_papers  t_sne_exaggerates_clusters_provably__related_work   
1  scientific_papers  t_sne_exaggerates_clusters_provably__related_work   
2  scientific_papers  t_sne_exaggerates_clusters_provably__related_work   
3  scientific_papers  t_sne_exaggerates_clusters_provably__related_work   
4  scientific_papers  t_sne_exaggerates_clusters_provably__related_work   

   sent_index                                           sentence  split  \
0           0  There are two notably distinct ways of context...  train   
1           1                          In a chronological sense.  train   
2           2  Confidence in the data visualizations produced...  train   
3           3  Some works argue that these methods have merit...  train   
4           4  Along with these algorithmic performance guara...  train   

   token_count  
0           16  
1            4  
2           20  
3           31  
4           2